In [1]:
import cudaq
from cudaq import spin

In [2]:
from RBM_surrogate import *
#from New_MCMC_Proposal import *
from get_conn import get_conn
from Sampling import *

In [3]:
import os
import numpy as np
#import netket as nk
import scipy as sp
import numpy.linalg as la
import scipy.linalg as spla
import time
import matplotlib.pyplot as plt
import json

In [4]:
N=8
M=8
D=0
beta=1

In [5]:
def string_to_spin(str):
    Op = 1
    for i in range(len(str)):
        if str[i]=='X': Op*=spin.x(i)
        elif str[i]=='Y': Op*=spin.y(i)
        elif str[i]=='Z': Op*=spin.z(i)
        elif str[i]=='I': continue
    #print(str, Op)
    return Op

def string_to_ops(str):
    Pauli_Ops = []
    Coeffs = []
    H = 0

    for line in str.strip().split('\n'):
        coeff_str, pauli_op = line.split('] ')
        coeff = complex(coeff_str.strip('['))  
        Coeffs.append(coeff)
        Pauli_Ops.append(pauli_op.strip())
        H += coeff * string_to_spin(pauli_op)

    #print("Pauli_Ops =", Pauli_Ops)
    #print("Coeffs =", Coeffs)

    return H, Pauli_Ops, Coeffs



In [6]:
with open("ZnO.txt", "r") as file:
    Ham_str = file.read()

Ham, _, _ = string_to_ops(Ham_str)

In [7]:
print(Ham.get_term_count())

185


In [8]:
# Adam optimizer hyperparameters
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

In [9]:
file = "Data/"

In [10]:
from scipy.sparse import csr_array, coo_matrix
from scipy.sparse.linalg import eigsh

Ham_sp = Ham.to_sparse_matrix()
data, row, col = Ham_sp
Ham_coo = coo_matrix((data, (row, col)), shape=(2**8, 2**8))
Ham_csr = csr_array(Ham_coo)

lmbd, _ = eigsh(Ham_csr, k=1, which='SA')

print("Smallest eigenvalue:", lmbd)

Smallest eigenvalue: [-1852.22917156]


In [11]:
def plot_log(E_hist, E_err_hist):
    #ya, iter = smpl.median_filter(np.real(E_hist))
    ya = np.real(E_hist)
    xa = np.arange(len(ya))
    ci = np.array(E_err_hist) #[iter]
    fig, ax = plt.subplots()
    ax.plot(ya)
    ax.fill_between(xa, (ya-ci), (ya+ci), color='r', alpha=.5)
    ax.set_ylabel("Energy")
    ax.set_xlabel("Epochs")
    plt.hlines(lmbd[0],0,len(xa)-1,color='r',linestyles='dashed')
    plt.show()

In [12]:
# #Calculating quantities
# def calc_quants(smpl,grad,H):
#   s = smpl.enum(N)
#   local_energies = smpl.local_energy(H,s)
#   kernels = smpl.kernel(s)
#   phi = smpl.prob(s)

#   derivative_op_diag = np.zeros((12,2**N),dtype=complex)
#   local_gradients = np.zeros((12,2**N),dtype=complex)
#   s_vec = np.reshape(s, (len(s),1,N))

#   da,db,dw,du,dd,dc = get_params(grad)
#   dw = np.reshape(dw,-1)

#   idx_list = np.array([np.argmin(np.abs(da)), np.argmax(np.abs(da)), N+np.argmin(np.abs(db)), N+np.argmax(np.abs(db)), N+M+np.argmin(np.abs(dw)), N+M+np.argmax(np.abs(dw))])
#   idx_list = np.concatenate((idx_list,idx_list+N+M+N*M+N*D+D+N*N))

#   for i in range(12):
#     idx = idx_list[i]
#     var, _, _ = smpl.map_idx_to_var(idx)
#     local_gradients[i] = smpl.local_gradient(H,s,idx)
#     derivative_op_diag[i] = smpl.derivative_operator(s,s_vec,idx)[:,0]


def save_data(X,smpl,g,seed,E,E_smpl,prob_dist,sample_list):
  with open(file+"X_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(X,(1,-1)))
  with open(file+"Poly_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(smpl.poly,(1,-1)))
  with open(file+"E_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, [E])
  with open(file+"E_smpl_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, [E_smpl])
  with open(file+"prob_dist_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(prob_dist,(1,-1)))
  with open(file+"prob_dist_N="+str(N)+"_g="+str(g)+"_seed="+str(seed)+".txt", "ab") as f:
    np.savetxt(f, np.reshape(sample_list,(1,-1)))


In [13]:
import tensorflow as tf

2025-04-10 18:10:58.873670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 18:10:58.922591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744308658.952237   36248 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744308658.962041   36248 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 18:10:59.012968: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [14]:
sample_size = 20000

seed = 1

E_hist=[]
E_smpl_hist=[]

rbm = RBM_surrogate(N=N,M=M,seed=seed,)
x = tf.Variable(rbm.X)

tm=time.time()

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=epsilon)

for epoch in range(40):
  rbm = RBM_surrogate(N,M,X=x.numpy())
  rbm.build_surrogate()
  #prob_dist, sample_list = Sampling(rbm, sample_size, burn=sample_size//10,)

  E = rbm.Energy_exact(Ham)
  #E_smpl = rbm.Energy_sampling(Ham,prob_dist)
  #print("Energy: ", E, "\t +/- \t", np.abs(E_smpl - E))

  grad_exact = rbm.grad_exact(Ham)
  #grad = rbm.grad_Sampling(Ham,prob_dist)

  optimizer.apply_gradients([(grad_exact, x)])

  E_hist.append(E)
  #E_smpl_hist.append(E_smpl)

print("\n\n\nEnergy: ",E,"\n Time",time.time()-tm, lmbd[0])

plot_log()

2025-04-10 18:11:02.372436: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


ValueError: not enough values to unpack (expected 2, got 1)

In [15]:
s = rbm.enum(N)

In [ ]:
s[0]

array([1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
import numpy as np
import cudaq

def apply_pauli(op, bitstring):
    new_bitstring = np.copy(bitstring)
    phase_factor = np.ones(len(bitstring), dtype=complex)

    for qubit_idx in range(len(op)):
        pauli_op = op[qubit_idx]

        if pauli_op == 'I':
            continue
        elif pauli_op == 'X':
            new_bitstring[:,qubit_idx] = - new_bitstring[:,qubit_idx]
        elif pauli_op == 'Y':
            new_bitstring[:,qubit_idx] = - new_bitstring[:,qubit_idx]
            phase_factor *= 1j*bitstring[:,qubit_idx]
        elif pauli_op == 'Z':
            phase_factor *= bitstring[:,qubit_idx]
        else:
            print("Invalid Pauli Operator!")
            break

    return new_bitstring, phase_factor


def get_conn(hamiltonian: cudaq.SpinOperator, bitstring: np.ndarray):  
    spin_list = []
    val_list = []

    def process_term(term):
        op = term.to_string(False).strip()  # Get Pauli op without coefficient
        coeff = term.get_coefficient()  # Extract the coefficient

        new_bitstring, phase_factor = apply_pauli(op, bitstring)

        spin_list.append(new_bitstring)
        val_list.append(phase_factor * coeff)

    hamiltonian.for_each_term(process_term)
    
    spin_list = np.transpose(spin_list, [1,0,2])
    val_list = np.transpose(val_list)

    return spin_list, val_list



In [25]:
t_spin, t_val = get_conn(Ham, s)

In [28]:
print(Ham.get_term_count())

185


In [29]:
np.shape(t_spin), np.shape(t_val)

((185, 256, 8), (185, 256))

In [ ]:
def process(term):
    term_str = term.to_string(False).strip()  # Get Pauli string without coefficient
    coeff = term.get_coefficient() 
    print(term_str, coeff)
    return 0

In [ ]:
Ham.for_each_term(process)

ZIIIZIII (0.07683278387268688+0j)
IIXZXIII (-0.1019000613458991+0j)
IIIIXYYX (0.0056436929661327195+0j)
IIIIIIZZ (0.05273479794160388+0j)
IZZIIIII (0.11401881140843803+0j)
IIXIXIII (-0.02552723726749953+0j)
IIXZXZII (-0.006773459408848079+0j)
IZIIIIII (0.3360615782898551+0j)
ZIIIIIZI (0.05261127280347765+0j)
IXZZZIZX (0.0008873162566754989+0j)
IIIXIXII (-0.006773459408848079+0j)
IIIZIZII (0.0509634789330711+0j)
IIXXIIYY (-0.00416045509125441+0j)
IZIIIZII (0.07683278387268688+0j)
ZIIZIIII (0.11401881140843803+0j)
IIIIXXYY (-0.0056436929661327195+0j)
YYIYYIII (-0.003228170101447683+0j)
IZXZXIII (-0.04442680638889496+0j)
IIIIIZII (0.12813725281196714+0j)
IIIYZYZI (0.0007045361116006638+0j)
XZXXZZZX (4.096584891408475e-05+0j)
IZIIIIIZ (0.05261127280347765+0j)
IZIYZYII (-0.04119863628744728+0j)
IIIIIIZI (0.022133837257015523+0j)
IYZZZZIY (-9.800605214997687e-06+0j)
IIYZZXXY (-0.0038876416863400914+0j)
YYXZZXII (-0.0032281701014476825+0j)
YYIIXXII (-0.002851621669446524+0j)
XYYZZXII (0.00322

In [ ]:
print(Ham)

[0.07683278387268688+0j] ZIIIZIII
[-0.1019000613458991+0j] IIXZXIII
[0.0056436929661327195+0j] IIIIXYYX
[0.05273479794160388+0j] IIIIIIZZ
[0.11401881140843803+0j] IZZIIIII
[-0.02552723726749953+0j] IIXIXIII
[-0.006773459408848079+0j] IIXZXZII
[0.3360615782898551+0j] IZIIIIII
[0.05261127280347765+0j] ZIIIIIZI
[0.0008873162566754989+0j] IXZZZIZX
[-0.006773459408848079+0j] IIIXIXII
[0.0509634789330711+0j] IIIZIZII
[-0.00416045509125441+0j] IIXXIIYY
[0.07683278387268688+0j] IZIIIZII
[0.11401881140843803+0j] ZIIZIIII
[-0.0056436929661327195+0j] IIIIXXYY
[-0.003228170101447683+0j] YYIYYIII
[-0.04442680638889496+0j] IZXZXIII
[0.12813725281196714+0j] IIIIIZII
[0.0007045361116006638+0j] IIIYZYZI
[4.096584891408475e-05+0j] XZXXZZZX
[0.05261127280347765+0j] IZIIIIIZ
[-0.04119863628744728+0j] IZIYZYII
[0.022133837257015523+0j] IIIIIIZI
[-9.800605214997687e-06+0j] IYZZZZIY
[-0.0038876416863400914+0j] IIYZZXXY
[-0.0032281701014476825+0j] YYXZZXII
[-0.002851621669446524+0j] YYIIXXII
[0.00322817010144